In [1]:
import geopandas as gpd
import pandas as pd

import seaborn as sns
import numpy as np
import math
import matplotlib

from sklearn.model_selection import GridSearchCV
from sklearn import ensemble

import ee
import geemap

In [3]:
ee.Authenticate()
ee.Initialize(project='ee-curuai')